<a href="https://colab.research.google.com/github/Cmput-414/PoinTr/blob/master/project_implementation_with_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Step 1: Set environments**



In [ ]:
##########################################################Step 1: Git clone code ##################################################
# 1. we clone the code from the github.
# 2. This will create a directory /content/pointr
!git -C /content/ clone https://github.com/Cmput-414/PoinTr.git pointr
########################################################################################################################################

##########################################################Step 2: Install conda ####################################################
# 1. Install conda environment for later use.
# 2. We will use conda to install a vitural environments with python 3.6
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.12.0-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.12.0-Linux-x86_64.sh -b -f -p /usr/local/
!conda create --no-default-packages -n myenv python=3.6 --yes
########################################################################################################################################

##########################################################Step 3: set virtual environment #########################################
# 1. After we set virtual environment， we activate it, check if we are in correct environment.
# 2. Then we install correct cuda version and pytorch version.
# 3. Then we will start to install some other packages. (This step will take some times, around 10 minutes.
!source activate myenv && which python && conda install pytorch==1.4.0 torchvision==0.5.0 cudatoolkit=10.0 -c pytorch -y
!source activate myenv && pip install ninja
!source activate myenv && git config --global url."https://".insteadOf git://
!source activate myenv && pip install "git+git://github.com/erikwijmans/Pointnet2_PyTorch.git#egg=pointnet2_ops&subdirectory=pointnet2_ops_lib"
!source activate myenv && pip install --upgrade https://github.com/unlimblue/KNN_CUDA/releases/download/0.1/KNN_CUDA-0.1-py3-none-any.whl
%cd /content/pointr
!source activate myenv && pip install -r requirements.txt
!source activate myenv && cd /content/pointr/extensions/chamfer_dist && python setup.py install --user
!source activate myenv && cd /content/pointr/extensions/cubic_feature_sampling && python setup.py install --user
!source activate myenv && cd /content/pointr/extensions/gridding && python setup.py install --user 
!source activate myenv && cd /content/pointr/extensions/gridding_loss && python setup.py install --user
########################################################################################################################################

### **Step 2: Set dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# After set environment, we will start set dataset
# In this project, we will use KITTI and 
##################################################Step 1: KITTI dataset ############################################################
#https://drive.google.com/file/d/1Vvci0uu-eAKsBrDEwJWWBP_r9qr_eHzI/view?usp=share_link

!cp -r '/content/drive/Shareddrives/Pointr/kitti/bboxes' /content/pointr/data/KITTI
!cp -r '/content/drive/Shareddrives/Pointr/kitti/cars' /content/pointr/data/KITTI
!cp -r '/content/drive/Shareddrives/Pointr/kitti/tracklets' /content/pointr/data/KITTI
########################################################################################################################################

##################################################Step 2: PCN dataset ##############################################################
# https://drive.google.com/file/d/1hHIoAW97HUsc2A9F159xutd0ajar1mqi/view?usp=share_link

!unzip /content/drive/Shareddrives/Pointr/ShapeNetCompletion.zip -d /content/pointr/data
!cp -r '/content/pointr/data/PCN/PCN.json' /content/pointr/data/ShapeNetCompletion
!cp -r '/content/pointr/data/PCN/category.txt' /content/pointr/data/ShapeNetCompletion
!rm -rf /content/pointr/data/PCN
!mv /content/pointr/data/ShapeNetCompletion /content/pointr/data/PCN
########################################################################################################################################

### **Step 3: Train code**
In this step: we will get some check points, then we can use these check points to do test. Finally, we can get visual result from the training result.

In [ ]:
%cd /content/pointr
# In this cell, we train modified PoinTr on PCN dataset. (Modified by Adding CNN)
# Epoch =10; batch size =16
# Around 4 hours
# The check point will save in to new direcory: ./experiments
!source activate myenv && which python && bash ./scripts/train.sh 0 --config ./cfgs/PCN_models/PoinTr.yaml --exp_name example

/content/pointr
/usr/local/envs/myenv/bin/python
+ GPUS=0
+ PY_ARGS='--config ./cfgs/PCN_models/PoinTr.yaml --exp_name example'
+ CUDA_VISIBLE_DEVICES=0
+ python main.py --config ./cfgs/PCN_models/PoinTr.yaml --exp_name example
2022-12-17 05:05:59,950 - PoinTr - INFO - Copy the Config file from ./cfgs/PCN_models/PoinTr.yaml to ./experiments/PoinTr/PCN_models/example/config.yaml
2022-12-17 05:05:59,951 - PoinTr - INFO - args.config : ./cfgs/PCN_models/PoinTr.yaml
2022-12-17 05:05:59,951 - PoinTr - INFO - args.launcher : none
2022-12-17 05:05:59,951 - PoinTr - INFO - args.local_rank : 0
2022-12-17 05:05:59,951 - PoinTr - INFO - args.num_workers : 4
2022-12-17 05:05:59,951 - PoinTr - INFO - args.seed : 0
2022-12-17 05:05:59,951 - PoinTr - INFO - args.deterministic : False
2022-12-17 05:05:59,951 - PoinTr - INFO - args.sync_bn : False
2022-12-17 05:05:59,951 - PoinTr - INFO - args.exp_name : example
2022-12-17 05:05:59,951 - PoinTr - INFO - args.start_ckpts : None
2022-12-17 05:05:59,951 -

In [ ]:
%cd /content/pointr
# In this cell, we train a Original PoinTr on KITTI dataset. (Do not modified)
# Epoch =10; batch size =16
# Around 46 minutes
# The check point will save in to new direcory: ./experiments
!source activate myenv && which python && bash ./scripts/train.sh 0 --config ./cfgs/KITTI_models/PoinTr.yaml --exp_name example

/content/pointr
/usr/local/envs/myenv/bin/python
+ GPUS=0
+ PY_ARGS='--config ./cfgs/KITTI_models/PoinTr.yaml --exp_name example'
+ CUDA_VISIBLE_DEVICES=0
+ python main.py --config ./cfgs/KITTI_models/PoinTr.yaml --exp_name example
2022-12-17 02:36:40,594 - PoinTr - INFO - Copy the Config file from ./cfgs/KITTI_models/PoinTr.yaml to ./experiments/PoinTr/KITTI_models/example/config.yaml
2022-12-17 02:36:40,594 - PoinTr - INFO - args.config : ./cfgs/KITTI_models/PoinTr.yaml
2022-12-17 02:36:40,594 - PoinTr - INFO - args.launcher : none
2022-12-17 02:36:40,594 - PoinTr - INFO - args.local_rank : 0
2022-12-17 02:36:40,594 - PoinTr - INFO - args.num_workers : 4
2022-12-17 02:36:40,594 - PoinTr - INFO - args.seed : 0
2022-12-17 02:36:40,594 - PoinTr - INFO - args.deterministic : False
2022-12-17 02:36:40,594 - PoinTr - INFO - args.sync_bn : False
2022-12-17 02:36:40,594 - PoinTr - INFO - args.exp_name : example
2022-12-17 02:36:40,594 - PoinTr - INFO - args.start_ckpts : None
2022-12-17 02:3

In [ ]:
%cd /content/pointr
# In this cell, we train modified PoinTr on KITTI dataset. (Modified by Adding CNN)
# Epoch =10; batch size =16
# Around 45 minutes.
# The check point will save in to new direcory: ./experiments
!source activate myenv && which python && bash ./scripts/train.sh 0 --config ./cfgs/KITTI_models/PoinTr.yaml --exp_name example

/content/pointr
/usr/local/envs/myenv/bin/python
+ GPUS=0
+ PY_ARGS='--config ./cfgs/KITTI_models/PoinTr.yaml --exp_name example'
+ CUDA_VISIBLE_DEVICES=0
+ python main.py --config ./cfgs/KITTI_models/PoinTr.yaml --exp_name example
Create experiment path successfully at ./experiments/PoinTr/KITTI_models/example
Create TFBoard path successfully at ./experiments/PoinTr/KITTI_models/TFBoard/example
2022-12-17 00:45:48,712 - PoinTr - INFO - Copy the Config file from ./cfgs/KITTI_models/PoinTr.yaml to ./experiments/PoinTr/KITTI_models/example/config.yaml
2022-12-17 00:45:48,712 - PoinTr - INFO - args.config : ./cfgs/KITTI_models/PoinTr.yaml
2022-12-17 00:45:48,712 - PoinTr - INFO - args.launcher : none
2022-12-17 00:45:48,712 - PoinTr - INFO - args.local_rank : 0
2022-12-17 00:45:48,712 - PoinTr - INFO - args.num_workers : 4
2022-12-17 00:45:48,712 - PoinTr - INFO - args.seed : 0
2022-12-17 00:45:48,712 - PoinTr - INFO - args.deterministic : False
2022-12-17 00:45:48,712 - PoinTr - INFO - a

### **Step 4: Test code**
In this step, we use training result to do test, and get some visual result.

In [ ]:
%cd /content/pointr
# In this cell, we test modified PoinTr on PCN dataset. (Modified by Adding CNN)
# Epoch =10; batch size =16
# Around 16 minutes
# The check point will save in to new direcory: ./experiments
!source activate myenv && which python && bash ./scripts/test.sh 0 --ckpts ./experiments/PoinTr/PCN_models/example/ckpt-best.pth --config ./cfgs/PCN_models/PoinTr.yaml --exp_name example

/content/pointr
/usr/local/envs/myenv/bin/python
+ GPUS=0
+ PY_ARGS='--ckpts ./experiments/PoinTr/PCN_models/example/ckpt-best.pth --config ./cfgs/PCN_models/PoinTr.yaml --exp_name example'
+ CUDA_VISIBLE_DEVICES=0
+ python main.py --test --ckpts ./experiments/PoinTr/PCN_models/example/ckpt-best.pth --config ./cfgs/PCN_models/PoinTr.yaml --exp_name example
Create experiment path successfully at ./experiments/PoinTr/PCN_models/test_example
Create TFBoard path successfully at ./experiments/PoinTr/PCN_models/TFBoard/test_example
2022-12-17 09:02:24,445 - PoinTr - INFO - Copy the Config file from ./cfgs/PCN_models/PoinTr.yaml to ./experiments/PoinTr/PCN_models/test_example/config.yaml
2022-12-17 09:02:24,445 - PoinTr - INFO - args.config : ./cfgs/PCN_models/PoinTr.yaml
2022-12-17 09:02:24,445 - PoinTr - INFO - args.launcher : none
2022-12-17 09:02:24,445 - PoinTr - INFO - args.local_rank : 0
2022-12-17 09:02:24,445 - PoinTr - INFO - args.num_workers : 4
2022-12-17 09:02:24,445 - PoinTr - I

In [ ]:
%cd /content/pointr
# In this cell, we test modified PoinTr on KITTI dataset. (Do not modified)
# Epoch =10; batch size =16
# Around 4 seconds
# The check point will save in to new direcory: ./experiments
!source activate myenv && which python && bash ./scripts/test.sh 0 --ckpts ./experiments/PoinTr/KITTI_models/example/ckpt-best.pth --config ./cfgs/KITTI_models/PoinTr.yaml --exp_name example

/content/pointr
/usr/local/envs/myenv/bin/python
+ GPUS=0
+ PY_ARGS='--ckpts ./experiments/PoinTr/KITTI_models/example/ckpt-best.pth --config ./cfgs/KITTI_models/PoinTr.yaml --exp_name example'
+ CUDA_VISIBLE_DEVICES=0
+ python main.py --test --ckpts ./experiments/PoinTr/KITTI_models/example/ckpt-best.pth --config ./cfgs/KITTI_models/PoinTr.yaml --exp_name example
2022-12-17 03:27:23,948 - PoinTr - INFO - Copy the Config file from ./cfgs/KITTI_models/PoinTr.yaml to ./experiments/PoinTr/KITTI_models/test_example/config.yaml
2022-12-17 03:27:23,948 - PoinTr - INFO - args.config : ./cfgs/KITTI_models/PoinTr.yaml
2022-12-17 03:27:23,948 - PoinTr - INFO - args.launcher : none
2022-12-17 03:27:23,948 - PoinTr - INFO - args.local_rank : 0
2022-12-17 03:27:23,948 - PoinTr - INFO - args.num_workers : 4
2022-12-17 03:27:23,948 - PoinTr - INFO - args.seed : 0
2022-12-17 03:27:23,948 - PoinTr - INFO - args.deterministic : False
2022-12-17 03:27:23,949 - PoinTr - INFO - args.sync_bn : False
2022-12

In [ ]:
%cd /content/pointr
# In this cell, we test modified PoinTr on KITTI dataset. (Modified by Adding CNN)
# Epoch =10; batch size =16
# Around 3 seconds
# The check point will save in to new direcory: ./experiments
!source activate myenv && which python && bash ./scripts/test.sh 0 --ckpts ./experiments/PoinTr/KITTI_models/example/ckpt-best.pth --config ./cfgs/KITTI_models/PoinTr.yaml --exp_name example

/content/pointr
/usr/local/envs/myenv/bin/python
+ GPUS=0
+ PY_ARGS='--ckpts ./experiments/PoinTr/KITTI_models/example/ckpt-best.pth --config ./cfgs/KITTI_models/PoinTr.yaml --exp_name example'
+ CUDA_VISIBLE_DEVICES=0
+ python main.py --test --ckpts ./experiments/PoinTr/KITTI_models/example/ckpt-best.pth --config ./cfgs/KITTI_models/PoinTr.yaml --exp_name example
Create experiment path successfully at ./experiments/PoinTr/KITTI_models/test_example
Create TFBoard path successfully at ./experiments/PoinTr/KITTI_models/TFBoard/test_example
2022-12-17 01:41:56,781 - PoinTr - INFO - Copy the Config file from ./cfgs/KITTI_models/PoinTr.yaml to ./experiments/PoinTr/KITTI_models/test_example/config.yaml
2022-12-17 01:41:56,781 - PoinTr - INFO - args.config : ./cfgs/KITTI_models/PoinTr.yaml
2022-12-17 01:41:56,781 - PoinTr - INFO - args.launcher : none
2022-12-17 01:41:56,781 - PoinTr - INFO - args.local_rank : 0
2022-12-17 01:41:56,781 - PoinTr - INFO - args.num_workers : 4
2022-12-17 01:41:5